In [ ]:
import re
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

In [ ]:
# Low pass filter to filter out unwanted noise from the force plate data.
def low_pass_filter(x, fc = 400, fs = 1000):
    w = fc / (fs / 2)  # Normalize the frequency
    b, a = signal.butter(3, w, "low")

    return signal.filtfilt(b, a, x, axis=0)

In [ ]:
# Read kinetic data from text file and manipulate it into each force plate etc.
#
# Fx = Data(:,1)+ Data(:,2);
# Fy = (Data (:,3) + Data(:,4));
# Fz = Data (:,5)+ Data (:,6) + Data (:,7) + Data (:,8);
#
# az0 = - 0.062;
# a = 0.210
# b = 0.350
#
# The Kistler force plate dimensions are 600 x 900 mm.
#
# Mx = b * (fz1 + fz2 - fz3 - fz4)
# My = a * (-fz1 + fz2 + fz3 - fz4)
# Mz = b * (-fx12 + fx34) + a * (fy14 - fy23)
# Mx' = Mx + Fy*az0
# My' = My - Fx*az0
# cop_x = -My' / Fz
# cop_y = Mx' / Fz
# Tz = Mz - Fy * ax + Fx * ay
# Declare constants used to convert the raw stream of data to metrics.
num_force_plates = 8
num_data_columns = 8
az0 = -0.062
a = 0.210
b = 0.350
txt_fname = "/Users/zico/OneDrive - University of Cape Town/msc/data/grf_kinetic_data/Arabia_070909_Trail06/Arabia_070909_06.txt"
with open(txt_fname, "r") as f:
    data_str = f.read()
    data = re.split("[\t\n]", data_str)
    data = np.array(data[:-1], dtype=np.float32)
    # TODO: There are meant to be 8 force plates but we only have data for 7 of them?
    force_plate_data = data.reshape((-1, num_force_plates, num_data_columns))
    # Run through each force plate and determine values
    df_list = []
    for i in range(num_force_plates):
        force_plate_data_filtered = low_pass_filter(force_plate_data[:, i], fc=400)
        fx12 = force_plate_data_filtered[:, 0]
        fx34 = force_plate_data_filtered[:, 1]
        fy14 = force_plate_data_filtered[:, 2]
        fy23 = force_plate_data_filtered[:, 3]
        fz1 = force_plate_data_filtered[:, 4]
        fz2 = force_plate_data_filtered[:, 5]
        fz3 = force_plate_data_filtered[:, 6]
        fz4 = force_plate_data_filtered[:, 7]
        Fx = fx12 + fx34
        Fy = fy14 + fy23
        Fz = fz1 + fz2 + fz3 + fz4
        Mx = b * (fz1 + fz2 - fz3 - fz4)
        My = a * (-fz1 + fz2 + fz3 - fz4)
        Mz = b * (-fx12 + fx34) + a * (fy14 - fy23)
        Mx_ = Mx + Fy*az0
        My_ = My - Fx*az0
        cop_x = -My_ / Fz
        cop_y = Mx_ / Fz
        Tz = Mz - Fy * cop_x + Fx * cop_y
        df_list.append(pd.DataFrame(np.array([Fx, Fy, Fz, cop_x, cop_y, Tz]).T,
                            columns=["Fx", "Fy", "Fz", "cop_x", "cop_y", "Tz"]))
    # Concatenate results and write to file.
    df = pd.concat(df_list, keys=[i for i in range(num_force_plates)], axis=0)
    df.index.set_names(["force_plate", "frame"], inplace=True)
    df.to_csv(
        os.path.join("/Users/zico/OneDrive - University of Cape Town/msc/data/grf_kinetic_data/Arabia_070909_Trail06",
                     "force_plate_data.csv"))
    df.to_hdf(os.path.join(
        "/Users/zico/OneDrive - University of Cape Town/msc/data/grf_kinetic_data/Arabia_070909_Trail06",
        "force_plate_data.h5"),
              "force_plate_data_df",
              format="table",
              mode="w")


In [ ]:
# Plotting results.
# The Kistler force plate dimensions are 600 x 900 mm.
x_axis_m = 0.6
y_axis_m = 0.9
out_dir = "/Users/zico/OneDrive - University of Cape Town/msc/data/grf_kinetic_data/Arabia_070909_Trail06"
fig = plt.figure(figsize=(16, 12), dpi=120)
ax = fig.add_subplot(111)
for i in range(num_force_plates):
    fp_data = df.query(f"force_plate == {i}")
    # fig = plt.figure(figsize=(16, 12), dpi=120)
    # axs = fig.subplots(1, 2).flatten()
    # axs[0].plot(fp_data["Fx"].values, color="b", label="Fx")
    # axs[0].plot(fp_data["Fy"].values, color="g", label="Fy")
    # axs[0].plot(fp_data["Fz"].values, color="r", label="Fz")
    # axs[0].set_title("GRF")
    # axs[0].set_xlabel("Frames")
    # axs[0].set_ylabel("Force (N)")
    # axs[0].legend()
    # axs[1].scatter(fp_data["cop_x"].values, fp_data["cop_y"].values)
    # axs[1].set_xlim((-x_axis_m, x_axis_m))
    # axs[1].set_ylim((-y_axis_m, y_axis_m))
    # axs[1].set_title("COP")
    # axs[1].set_xlabel("X")
    # axs[1].set_ylabel("Y")
    # plt.savefig(os.path.join(out_dir, f"force_plate_{i+1}.png"))
    # plt.close()
    x_vals = fp_data.index.get_level_values(1).values
    ax.plot(x_vals, fp_data["Fx"].values, color="b", label="Fx")
    ax.plot(x_vals, fp_data["Fy"].values, color="g", label="Fy")
    ax.plot(x_vals, fp_data["Fz"].values, color="r", label="Fz")
    ax.set_title("GRF")
    ax.set_xlabel("Frames")
    ax.set_ylabel("Force (N)")
    ax.set_xlim((0, 3000))
    ax.legend(["Fx", "Fy", "Fz"])
plt.savefig(os.path.join(out_dir, f"grf.png"))
plt.close()
